In [1]:
import os

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Bokeh
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis, Spectral8, Reds, Greens, Category20
from bokeh.models import HoverTool, CategoricalColorMapper

output_notebook() 

Loading BokehJS ...

In [2]:
DT = 0.1 # Simulation parameter

KC = 0.16
KH = 0.0896
VF = 25.0
W = 6.25
E  = 25.0*0.3
GCAV = 1/(KC*W)
GHDV = 1/(KH*W)
SCAV = VF/(KC*W)+1/KC
SHDV = VF/(KH*W)+1/KH
dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}
U_MAX = 1.5
U_MIN = -1.5

In [3]:
dir_path = os.getcwd()
engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

try: 
    ltbstr = 'Loaded table in: '
    connection = engine.connect()    
    traj = Table('traj', metadata, autoload=True, autoload_with=engine)
    closed = Table('closed', metadata, autoload=True, autoload_with=engine)
    headway = Table('headway', metadata, autoload=True, autoload_with=engine)
    control = Table('control', metadata, autoload=True, autoload_with=engine)    
except:
    ltbstr = 'Table could not be loaded: '
finally: 
    print(ltbstr, engine)

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


In [4]:
stmt = select([closed])
print(stmt)
results = connection.execute(stmt).fetchall()

stmt = select([traj])
print(stmt)
resultsOL = connection.execute(stmt).fetchall()

SELECT closed.ti, closed.id, closed.type, closed.tron, closed.voie, closed.dst, closed.abs, closed.vit, closed.ldr, closed.spc, closed.vld, closed.ctr 
FROM closed
SELECT traj.ti, traj.id, traj.type, traj.tron, traj.voie, traj.dst, traj.abs, traj.vit, traj.ldr, traj.spc, traj.vld 
FROM traj


In [5]:
column_names = closed.columns.keys()
trajCLDf = pd.DataFrame(results, columns = column_names)

column_names = traj.columns.keys()
trajOLDf = pd.DataFrame(resultsOL, columns = column_names)

trajCLDf.head()

,ti,id,type,tron,voie,dst,abs,vit,ldr,spc,vld,ctr
0,1.3,0,CAV,In_main,1,1.25,-998.75,25.0,0,31.25,25.0,None
1,1.4,0,CAV,In_main,1,3.75,-996.25,25.0,0,31.25,25.0,None
2,1.5,0,CAV,In_main,1,6.25,-993.75,25.0,0,31.25,25.0,None
3,1.6,0,CAV,In_main,1,8.75,-991.25,25.0,0,31.25,25.0,None
4,1.7,0,CAV,In_main,1,11.25,-988.75,25.0,0,31.25,25.0,None


In [6]:
trajCLDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5289 entries, 0 to 5288
Data columns (total 12 columns):
ti      5289 non-null float64
id      5289 non-null int64
type    5289 non-null object
tron    5289 non-null object
voie    5289 non-null int64
dst     5289 non-null float64
abs     5289 non-null float64
vit     5289 non-null float64
ldr     5289 non-null int64
spc     5289 non-null float64
vld     5289 non-null float64
ctr     0 non-null object
dtypes: float64(6), int64(3), object(3)
memory usage: 495.9+ KB


In [7]:
trajHDVDf = trajOLDf[trajOLDf['type']=='HDV']

trajCLDf['id'] = trajCLDf['id'].astype('str')
trajHDVDf['id'] = trajHDVDf['id'].astype('str')

sourceCL = ColumnDataSource(trajCLDf)
sourceHDV = ColumnDataSource(trajHDVDf)

CAV_list = trajCLDf.id.unique().tolist()
HDV_list = trajHDVDf.id.unique().tolist()

CAV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 ==0 ]
HDV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 !=0 ]

color_mapper_CAV = CategoricalColorMapper(factors=CAV_list, palette=CAV_colors)
color_mapper_HDV = CategoricalColorMapper(factors=HDV_list, palette=HDV_colors)

colorCAV = dict(field='id', transform=color_mapper_CAV)
colorHDV = dict(field='id', transform=color_mapper_HDV)

hover = HoverTool(tooltips=[('t','@ti'),('x','@abs'),('type','@type'), ('id','@id')])
hoverRef = HoverTool(tooltips=[('t','@ti'),('x','@abs'), ('id','@id')])
hoverCtr = HoverTool(tooltips=[('t','@ti'),('u','@ctr'), ('id','@id')])

/Users/ladino/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
p = figure(title = "Headspace", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom, reset'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Spacing [m]'

# Spacing 

p.circle('ti', 'spc', source=sourceCL, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.add_tools(hover)
show(p)

#
#trajDf.plot(kind='scatter',x='ti',y ='spc', c = trajDf.tron.map(colormap), figsize = (7.5, 7.5), s = 2, grid = True);

In [9]:
stmt = select([headway])
print(stmt)
results = connection.execute(stmt).fetchall()

SELECT headway.ti, headway.id, headway.gapt 
FROM headway


In [10]:
column_names = headway.columns.keys()
headDf = pd.DataFrame(results, columns = column_names)
headDf.head()

,ti,id,gapt
0,1.3,0,1.0
1,1.4,0,1.0
2,1.5,0,1.0
3,1.6,0,1.0
4,1.7,0,1.0


In [11]:
# Control performance 

p = figure(title = "Time Gap", 
           width=700, 
           height=700, 
          tools = ['lasso_select', 'wheel_zoom'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Tau [s]'

trajCLDf['tau'] = (trajCLDf['spc']-1/KC).divide(trajCLDf['vit'])
headDf['id'] = headDf['id'].astype('str')

REF_list = headDf.id.unique().tolist()

REF_colors = [x for i, x in enumerate(Category20[16]) if i % 2 !=0]

color_mapper_REF = CategoricalColorMapper(factors=REF_list, palette=REF_colors)

sourceTau = ColumnDataSource(trajCLDf)
sourceRef = ColumnDataSource(headDf)

colorREF = dict(field='id', transform=color_mapper_REF)

p.circle('ti', 'tau', source=sourceTau, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.circle('ti', 'gapt', source=sourceRef, color = colorREF, selection_color = colorREF, size = 3, nonselection_alpha=0.1)
p.add_tools(hoverRef)
show(p)

#
#trajDf.plo

In [12]:
stmt = select([control])
print(stmt)
results = connection.execute(stmt).fetchall()

SELECT control.ti, control.id, control.type, control.tron, control.voie, control.ctr 
FROM control


In [13]:
column_names = control.columns.keys()
controlDf = pd.DataFrame(results, columns = column_names)
controlDf.head()

,ti,id,type,tron,voie,ctr
0,1.3,0,CAV,In_main,1,0.0
1,1.4,0,CAV,In_main,1,0.0
2,1.5,0,CAV,In_main,1,0.0
3,1.6,0,CAV,In_main,1,0.0
4,1.7,0,CAV,In_main,1,0.0


In [14]:
controlDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5274 entries, 0 to 5273
Data columns (total 6 columns):
ti      5274 non-null float64
id      5274 non-null int64
type    5274 non-null object
tron    5274 non-null object
voie    5274 non-null int64
ctr     5274 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 247.3+ KB


In [15]:
p = figure(title = "Control signal", 
           width=700, 
           height=700, 
          tools = ['lasso_select', 'wheel_zoom, reset'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Acceleration [ms-2]'

controlDf['id'] = controlDf['id'].astype('str')

sourceCTRL = ColumnDataSource(controlDf)

p.circle('ti', 'ctr', source=sourceCTRL, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.add_tools(hoverCtr)
show(p)

In [16]:
#colors = [colormapid[x] for x in trajCLDf.id]


p = figure(title = "Trajectories", 
           width=700, 
           height=700, 
          tools = ['lasso_select', 'wheel_zoom, reset'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'X [m]'

p.circle('ti', 'abs', source=sourceHDV, color = colorHDV, selection_color = colorHDV, size = 3, nonselection_alpha=0.1)
p.circle('ti', 'abs', source=sourceCL, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
# p.circle('ti', 'abs', source=sourceHDV, color = 'salmon', size = 3)
# p.circle('ti', 'abs', source=sourceCL, color = 'deepskyblue', size = 3)
p.add_tools(hover)

show(p)
